## Atheism subreddit - check vectorization and modelling

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [2]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?code_challenge=auThI0LAtQL2Vw7qYLJP9ObrghLCxUL6IhDDa-gGo4M&prompt=select_account&code_challenge_method=S256&access_type=offline&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&client_id=32555940559.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth


Enter verification code: 4/tgFLgVP6jXXHxiBk5AM6bDwkwI7ACJXnHQ5tN780R8Gave-QDX2tD-U
If you need to use ADC, see:
  gcloud auth application-default --help

You are now logged in as [galli.giuly@gmail.com].
Your current project is [reddit-master].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [3]:
!gcloud config set project reddit-master

Updated property [core/project].


In [0]:
# !pip install wordcloud

In [87]:
import pandas as pd
import numpy as np
import pickle
import nltk.data
import nltk
import ast
nltk.download('punkt')

import matplotlib.pyplot as plt
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from gensim.models import FastText
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_score, classification_report, confusion_matrix

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Below, some intent of upload the full file and work on it.. it failed because of the dimension of the "all subreddits" file.

In [0]:
# !gsutil cp gs://reddit_final_results/comments_posts_tokenized_V2.csv .

In [59]:
# ! head comments_posts_tokenized_V2.csv

,subreddit,body,subreddit_id
0,aww,doubl multipl sub approach good ya mate,3
1,aww,2nd meatbal serious chunki english bulldog offici sir duke meatbal duke,3
2,aww,thought,3
3,aww,friend eat friend,3
4,aww,mayb doctor wait heart room,3
5,aww,tell want stay hell shave,3
6,aww,mean,3
7,aww,awesom,3
8,aww,san diego poster miss,3


In [0]:
# comments_posts_tokenized_df = pd.read_csv("comments_posts_tokenized_V2.csv", index_col=[0])

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
# !gsutil cp gs://reddit_final_results/comments_posts_tokenized.csv .

Copying gs://reddit_final_results/comments_posts_tokenized.csv...
/ [1 files][  1.7 GiB/  1.7 GiB]   83.4 MiB/s                                   
Operation completed over 1 objects/1.7 GiB.                                      


In [8]:
# comments_posts_tokenized_df = pd.read_csv("comments_posts_tokenized.csv", index_col=[0])

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [9]:
# Asfer a bad vectorization I've looked into the column body: the type is string and the vectorization process didn't work as I needed

# type(comments_posts_tokenized_df['body'].iloc[0])

str

In [0]:
# let's convert the column body into a list
# trying on a sample
# sample = comments_posts_tokenized_df.sample(20)
# sample['body'] = sample['body'].map(lambda x: ast.literal_eval(x))

In [11]:
# it works

# type(sample['body'].iloc[0])

list

In [0]:
# let's do it in our original df

 #comments_posts_tokenized_df['body'] = comments_posts_tokenized_df['body'].map(lambda x: ast.literal_eval(x))

In [6]:
# !gsutil cp gs://reddit_final_results/comments_posts_tokenized_df_gzip.pkl .

Copying gs://reddit_final_results/comments_posts_tokenized_df_gzip.pkl...
/ [1 files][488.7 MiB/488.7 MiB]                                                
Operation completed over 1 objects/488.7 MiB.                                    


From here I changed the strategy

In [0]:
subreddits = ['aww', 'nba', 'movies', 'todayilearned', 'IAmA', 'Fitness', 'worldnews', 'technology', 'europe', 'politics', 'atheism','science', 'funny', 'gaming']

In [7]:
for subreddit in subreddits:
    !gsutil cp gs://reddit_final_results/comments_posts_{subreddit}.pkl .
    print("comments_posts_" + subreddit + ".pkl downloaded")


Copying gs://reddit_final_results/comments_posts_aww.pkl...
\ [1 files][ 63.8 MiB/ 63.8 MiB]                                                
Operation completed over 1 objects/63.8 MiB.                                     
comments_posts_aww.pkl downloaded
Copying gs://reddit_final_results/comments_posts_nba.pkl...
\ [1 files][ 88.3 MiB/ 88.3 MiB]                                                
Operation completed over 1 objects/88.3 MiB.                                     
comments_posts_nba.pkl downloaded
Copying gs://reddit_final_results/comments_posts_movies.pkl...
\ [1 files][103.2 MiB/103.2 MiB]                                                
Operation completed over 1 objects/103.2 MiB.                                    
comments_posts_movies.pkl downloaded
Copying gs://reddit_final_results/comments_posts_todayilearned.pkl...
| [1 files][101.1 MiB/101.1 MiB]                                                
Operation completed over 1 objects/101.1 MiB.                           

In [5]:
!ls

adc.json		    comments_posts_movies.pkl
comments_posts_atheism.pkl  comments_posts_nba.pkl
comments_posts_aww.pkl	    comments_posts_politics.pkl
comments_posts_europe.pkl   comments_posts_science.pkl
comments_posts_Fitness.pkl  comments_posts_technology.pkl
comments_posts_funny.pkl    comments_posts_todayilearned.pkl
comments_posts_gaming.pkl   comments_posts_worldnews.pkl
comments_posts_IAmA.pkl     sample_data


I decided to try my validations and models on one subreddit `atheism` just to check fix errors and check if I'm following the corrects steps.

### comments_posts_atheism

In [0]:
df_atheism = pd.read_pickle("/content/comments_posts_atheism.pkl")

In [7]:
df_atheism.columns

Index(['Unnamed: 0.1', 'subreddit', 'body', 'subreddit_id'], dtype='object')

In [8]:
df_atheism.head(3)

,Unnamed: 0.1,subreddit,body,subreddit_id
88650,88650,atheism,"[argu, deep, human, except, thing, defianc, idea]",2
88651,88651,atheism,"[differ, harri, potter, bibl, coupl, billion, ...",2
88652,88652,atheism,"[explain, argument]",2


In [9]:
type(df_atheism['body'].iloc[0])

list

Let's rock!

## Train, validation, test

In [17]:
subreddits = df_atheism['subreddit'].unique()
print(subreddits)

['atheism']


In [0]:
X = df_atheism['body']
y = df_atheism['subreddit_id']

In [0]:
# Definint a fucntion that slit the dataset into three subsets: train, val and test
def train_dev_test_split(X, y):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=42, stratify=y_val)
    return dict(X_train=X_train, 
                X_val=X_val, 
                X_test=X_test, 
                y_train=y_train, 
                y_val=y_val, 
                y_test=y_test)

In [0]:
df_atheism_split = train_dev_test_split(X,y)

In [21]:
df_atheism_split.keys()

dict_keys(['X_train', 'X_val', 'X_test', 'y_train', 'y_val', 'y_test'])

## Vectorization

#### FTIDF vectorization - atheism

In [22]:
%%time
tfidf=TfidfVectorizer()
tfidf_atheism= tfidf.fit_transform(df_atheism['body'].astype(str))


CPU times: user 19.9 s, sys: 296 ms, total: 20.2 s
Wall time: 20.1 s


In [30]:
tfidf.get_feature_names()[::-1]

['zzzzzzzzzzzzzz',
 'zzzzzzzzzzz',
 'zzzzzzzzxzzzzzzzzz',
 'zzzzzz',
 'zzz',
 'zzshzzshhhzzsshhhhi',
 'zzrt',
 'zz9',
 'zz',
 'zyx',
 'zyrgfrh',
 'zyprexa',
 'zyklonb',
 'zyklon',
 'zyhem',
 'zyhe',
 'zygoteyou',
 'zygotethat',
 'zygotesof',
 'zygotesin',
 'zygoteshowev',
 'zygotesgti',
 'zygotesgtalso',
 'zygotesfetus',
 'zygotesembryosedit',
 'zygotesblastula',
 'zygoteor',
 'zygotenobodi',
 'zygoteit',
 'zygotei',
 'zygotefoetus',
 'zygotefetusembryoetc',
 'zygotefetus',
 'zygoteembryofetusbabi',
 'zygoteembryofetus',
 'zygoteembryo',
 'zygotecreampi',
 'zygotebut',
 'zygotea',
 'zygote',
 'zygot',
 'zyclon',
 'zy',
 'zxcvb94105',
 'zx',
 'zwoodfin03it',
 'zwischen',
 'zwinglist',
 'zwing',
 'zwicker',
 'zwhat',
 'zwell',
 'zweiundvierzig',
 'zweihand',
 'zwei',
 'zweebil',
 'zweckbindung',
 'zwart',
 'zwaanof',
 'zwaan',
 'zvp',
 'zvolili',
 'zvk',
 'zvanth',
 'zuul',
 'zutreten',
 'zustnd',
 'zustimmen',
 'zuss',
 'zus',
 'zurvan',
 'zurita',
 'zurishaddaigt',
 'zurich',
 'zurg',


In [23]:
v = tfidf_atheism[0, :].toarray()

# compute the most common value
c = Counter(v[0])
print(c.most_common(1))

print(f"min: {v.min()}")
print(f"avg: {v.mean()}")
print(f"max: {v.max()}")
print(f"std: {v.std()}")
print(f"mode: {c.most_common(1)}")

[(0.0, 368448)]
min: 0.0
avg: 6.787677600650915e-06
max: 0.604656600280137
std: 0.0016474190610529808
mode: [(0.0, 368448)]


### Count vectorizer - atheism

In [0]:
%%time 

bag_of_words_atheism = nltk.word_tokenize(df_atheism['body'].to_string())
bag_of_words_atheism = list(dict.fromkeys(bag_of_words_atheism))
print(bag_of_words_atheism[:10])
print(len(bag_of_words_atheism)) 

In [0]:
count_vectors_atheism = CountVectorizer(ngram_range=(1,2), strip_accents="unicode", encoding='utf-8')

train_term_count_autheism = count_vectors.fit_transform(df_split['X_train'].astype(str))
test_term_count_autheism = count_vectors.transform(df_split['X_val'].astype(str))
test_term_count_autheism = count_vectors.transform(df_split['X_val'].astype(str))

train_term_count_autheism

<7583345x27033690 sparse matrix of type '<class 'numpy.int64'>'
	with 200463859 stored elements in Compressed Sparse Row format>

In [0]:
count_vectors_atheism.get_feature_names()

['00',
 '00 00',
 '00 000',
 '00 0030',
 '00 01',
 '00 01100',
 '00 02',
 '00 04',
 '00 06',
 '00 10',
 '00 100',
 '00 11',
 '00 12',
 '00 1214',
 '00 16',
 '00 18',
 '00 190',
 '00 1900',
 '00 1i',
 '00 1just',
 '00 20',
 '00 207',
 '00 21',
 '00 22',
 '00 24',
 '00 2d',
 '00 310',
 '00 350',
 '00 3ellington',
 '00 40',
 '00 411',
 '00 610',
 '00 80',
 '00 90s',
 '00 91',
 '00 99',
 '00 absolut',
 '00 actual',
 '00 add',
 '00 agent',
 '00 aim',
 '00 alex',
 '00 alot',
 '00 american',
 '00 amv',
 '00 annoy',
 '00 anymor',
 '00 apr',
 '00 astonish',
 '00 aw',
 '00 axe',
 '00 ayrshir',
 '00 bad',
 '00 basic',
 '00 battl',
 '00 bbc',
 '00 believ',
 '00 benzodiazepin',
 '00 best',
 '00 better',
 '00 birth',
 '00 blkpg',
 '00 bob',
 '00 boost',
 '00 bpmprigioni',
 '00 breakthrough',
 '00 buck',
 '00 buckshot',
 '00 byte',
 '00 calori',
 '00 came',
 '00 camri',
 '00 cap',
 '00 caputo',
 '00 carb',
 '00 carreer',
 '00 case',
 '00 casualti',
 '00 caus',
 '00 center',
 '00 chanc',
 '00 charg',


In [0]:
v = train_term_count_atheism[0, :].toarray()

# compute the most common value
c = Counter(v[0])
print(c.most_common(1))

print(f"min: {v.min()}")
print(f"avg: {v.mean()}")
print(f"max: {v.max()}")
print(f"std: {v.std()}")
print(f"mode: {c.most_common(1)}")

[(0, 27033689)]
min: 0
avg: 3.6990880638196264e-08
max: 1
std: 0.00019233013094643186
mode: [(0, 27033689)]


In [0]:
list(count_vectors.vocabulary_.keys())[:10]

['think',
 'submiss',
 'removedviol',
 'rule',
 'thread',
 'answer',
 'wiki',
 'search',
 'rfit',
 'googl']

In [0]:
count_vectors_atheism.wv.most_similar("pope")

### FastText vectorization - atheism

In [26]:
# Let us start by downloading the most recent release

!wget https://github.com/facebookresearch/fastText/archive/v0.9.1.zip
!unzip v0.9.1.zip

--2019-11-23 22:05:34--  https://github.com/facebookresearch/fastText/archive/v0.9.1.zip
Resolving github.com (github.com)... 192.30.253.112
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/facebookresearch/fastText/zip/v0.9.1 [following]
--2019-11-23 22:05:34--  https://codeload.github.com/facebookresearch/fastText/zip/v0.9.1
Resolving codeload.github.com (codeload.github.com)... 140.82.113.9
Connecting to codeload.github.com (codeload.github.com)|140.82.113.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘v0.9.1.zip’

v0.9.1.zip              [  <=>               ]   4.13M  10.4MB/s    in 0.4s    

2019-11-23 22:05:35 (10.4 MB/s) - ‘v0.9.1.zip’ saved [4327207]

Archive:  v0.9.1.zip
b5b7d307274ce00ef52198fbc692ed3bd11d9856
   creating: fastText-0.9.1/
   creating: fastText-0.9.1/.circleci/
  inflating: fastTe

In [27]:
%cd fastText-0.9.1
# for command line tool :
!make
# for python bindings :
!pip install .

/content/fastText-0.9.1
c++ -pthread -std=c++0x -march=native -O3 -funroll-loops -DNDEBUG -c src/args.cc
c++ -pthread -std=c++0x -march=native -O3 -funroll-loops -DNDEBUG -c src/matrix.cc
c++ -pthread -std=c++0x -march=native -O3 -funroll-loops -DNDEBUG -c src/dictionary.cc
c++ -pthread -std=c++0x -march=native -O3 -funroll-loops -DNDEBUG -c src/loss.cc
c++ -pthread -std=c++0x -march=native -O3 -funroll-loops -DNDEBUG -c src/productquantizer.cc
c++ -pthread -std=c++0x -march=native -O3 -funroll-loops -DNDEBUG -c src/densematrix.cc
c++ -pthread -std=c++0x -march=native -O3 -funroll-loops -DNDEBUG -c src/quantmatrix.cc
c++ -pthread -std=c++0x -march=native -O3 -funroll-loops -DNDEBUG -c src/vector.cc
c++ -pthread -std=c++0x -march=native -O3 -funroll-loops -DNDEBUG -c src/model.cc
c++ -pthread -std=c++0x -march=native -O3 -funroll-loops -DNDEBUG -c src/utils.cc
c++ -pthread -std=c++0x -march=native -O3 -funroll-loops -DNDEBUG -c src/meter.cc
c++ -pthread -std=c++0x -march=native -O3 -fun

In [28]:
sentences_text = [word for word in df_atheism['body'].values]
sentences_text[:3]

[['argu', 'deep', 'human', 'except', 'thing', 'defianc', 'idea'],
 ['differ',
  'harri',
  'potter',
  'bibl',
  'coupl',
  'billion',
  'peopl',
  'believ',
  'harri',
  'potter',
  'fiction',
  '23',
  'billion',
  'peopl',
  'believ',
  'bibl',
  'truth',
  'figur',
  'actual',
  'peopl',
  'believ',
  'import',
  'instanc',
  'exist',
  'justif',
  'bibl',
  'believ',
  'jesus',
  'die',
  'fact',
  'jewish',
  'pharise',
  'save',
  'life',
  'book',
  'liter',
  'saysi',
  'sure',
  'feel',
  'mind',
  'fuck',
  'have',
  'accept',
  'truth',
  'jew',
  'christian',
  'muslim',
  'atheist',
  'regardless',
  'fact',
  'liter',
  'bibl',
  'say',
  'stop',
  'believ',
  'lie',
  'spread',
  'christian',
  ' ',
  'ps',
  ' ',
  '60',
  'popul',
  'believ',
  'jesus',
  'way',
  'democrat',
  'major',
  'planet',
  'figur',
  'accord',
  'sourc',
  'believ',
  'obvious',
  'import',
  'doubt',
  'need',
  'quot',
  'sourc',
  'critic',
  'atheist'],
 ['explain', 'argument']]

In [29]:
sentences_text[0]

['argu', 'deep', 'human', 'except', 'thing', 'defianc', 'idea']

Now we can train our classifier

In [30]:
%%time
model_fasttext_atheism = FastText(sentences_text, size=200, window=5, min_count=5, workers=3,sg=1)
# model_fasttext_atheism.save("model_fasttext_atheism")
# !gsutil cp /content/model_fasttext_atheism gs://reddit_models/

CPU times: user 27min 30s, sys: 1.73 s, total: 27min 32s
Wall time: 9min 30s


In [19]:
!pwd

/content/fastText-0.9.1


In [20]:
!gsutil cp /content/fastText-0.9.1/model_fasttext_atheism gs://reddit_models/

Copying file:///content/fastText-0.9.1/model_fasttext_atheism [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

\
Operation completed over 1 objects/189.0 MiB.                                    


In [31]:
model_fasttext_atheism.wv.vectors.shape

(42016, 200)

In [32]:
model_fasttext_atheism.wv.most_similar("pope")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('popei', 0.7969644069671631),
 ('franci', 0.7832726836204529),
 ('gtpope', 0.7675614953041077),
 ('popey', 0.7129794359207153),
 ('vatican', 0.704525351524353),
 ('papal', 0.650409460067749),
 ('papaci', 0.646502673625946),
 ('jp2', 0.6357030868530273),
 ('benedict', 0.6325761079788208),
 ('ratzing', 0.6322858333587646)]

In [43]:
model_fasttext_atheism.wv.doesnt_match("pope")

/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'o'

In [48]:
model_fasttext_atheism.wv.similarity("pope","hairdryer")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.16275942

In [49]:
model_fasttext_atheism.wv.similarity("pope","priest")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.51841784

### Word2vec - atheism

In [0]:
EMBEDDING_DIM = 200

In [38]:
sentences_w2v_atheism = [word for word in df_atheism['body'].values]
sentences_w2v_atheism[:3]

[['argu', 'deep', 'human', 'except', 'thing', 'defianc', 'idea'],
 ['differ',
  'harri',
  'potter',
  'bibl',
  'coupl',
  'billion',
  'peopl',
  'believ',
  'harri',
  'potter',
  'fiction',
  '23',
  'billion',
  'peopl',
  'believ',
  'bibl',
  'truth',
  'figur',
  'actual',
  'peopl',
  'believ',
  'import',
  'instanc',
  'exist',
  'justif',
  'bibl',
  'believ',
  'jesus',
  'die',
  'fact',
  'jewish',
  'pharise',
  'save',
  'life',
  'book',
  'liter',
  'saysi',
  'sure',
  'feel',
  'mind',
  'fuck',
  'have',
  'accept',
  'truth',
  'jew',
  'christian',
  'muslim',
  'atheist',
  'regardless',
  'fact',
  'liter',
  'bibl',
  'say',
  'stop',
  'believ',
  'lie',
  'spread',
  'christian',
  ' ',
  'ps',
  ' ',
  '60',
  'popul',
  'believ',
  'jesus',
  'way',
  'democrat',
  'major',
  'planet',
  'figur',
  'accord',
  'sourc',
  'believ',
  'obvious',
  'import',
  'doubt',
  'need',
  'quot',
  'sourc',
  'critic',
  'atheist'],
 ['explain', 'argument']]

In [40]:
%%time

modelWV_atheism = Word2Vec(sentences_w2v_atheism, workers = 3, min_count=5, window = 10, size = EMBEDDING_DIM)
modelWV_atheism.train(sentences_w2v_atheism, total_examples=len(sentences_w2v_atheism), epochs=50)
# modelWV_atheism.save("model_word2vec")
# !gsutil cp /content/model_word2vec gs://reddit_models/

CPU times: user 52min 19s, sys: 8.85 s, total: 52min 27s
Wall time: 18min 11s


In [41]:
modelWV_atheism.wv.vectors.shape

(42016, 200)

In [42]:
modelWV_atheism.wv.most_similar("pope")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('franci', 0.7146515250205994),
 ('vatican', 0.6533546447753906),
 ('ratzing', 0.5700125694274902),
 ('gtpope', 0.561491072177887),
 ('papaci', 0.5403313636779785),
 ('papal', 0.526868462562561),
 ('vigano', 0.5170674324035645),
 ('jp2', 0.514457643032074),
 ('cardin', 0.5137941837310791),
 ('benedict', 0.5112627744674683)]

In [50]:
modelWV_atheism.wv.similarity("pope","hairdryer")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.02832523

In [51]:
modelWV_atheism.wv.similarity("pope","priest")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.4967405

## Modelling - atheism

The splitted df into train, validation and test is `df_atheism_split`. Let's using it for train the models

#### XGBClassifier

In [59]:
model = Pipeline([
    ("vectorize", TfidfVectorizer(ngram_range=(1,2), lowercase=False, preprocessor=' '.join)),
    ("classifier", XGBClassifier())
])

model.fit(df_atheism_split['X_train'], df_atheism_split['y_train'])

Pipeline(memory=None,
         steps=[('vectorize',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=False, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 2), norm='l2',
                                 preprocessor=<built-in method join of str object at 0x7f44a9f4f538>,
                                 smooth_idf=True, stop_words=None,
                                 strip...
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.1,
                               max_delta_step=0, max_depth=3,
                               min_child_weight=1, m

In [0]:
prediction = model.predict(df_atheism_split['X_val'])

In [63]:
prediction

array([2, 2, 2, ..., 2, 2, 2])

In [96]:
print ('accuracy ', accuracy_score(df_atheism_split['y_val'], prediction))
print ('precision ', precision_score(df_atheism_split['y_val'], prediction))
print (classification_report(df_atheism_split['y_val'], prediction))
print (confusion_matrix(df_atheism_split['y_val'], prediction))

accuracy  1.0
precision  0.0
              precision    recall  f1-score   support

           2       1.00      1.00      1.00    116040

    accuracy                           1.00    116040
   macro avg       1.00      1.00      1.00    116040
weighted avg       1.00      1.00      1.00    116040



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[[116040]]


In [75]:
cross_val_score(model,  df_atheism_split['X_val'], df_atheism_split['y_val'], cv=5,scoring="accuracy")

array([1., 1., 1., 1., 1.])

#### SVC

In [81]:
%%time

SVC_model = Pipeline([
                ("tfidf", TfidfVectorizer(ngram_range=(1,2), lowercase=False, preprocessor=' '.join)),
                ("clf", OneVsRestClassifier(LinearSVC(), n_jobs=1)),
            ])

SVC_model.fit(df_atheism_split['X_train'], df_atheism_split['y_train'])

CPU times: user 47.3 s, sys: 575 ms, total: 47.9 s
Wall time: 47.8 s


/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:76: UserWarning: Label not 2 is present in all training examples.
  str(classes[c]))


In [84]:
prediction_SVC = SVC_model.predict(df_atheism_split['X_val'])
prediction_SVC

array([2, 2, 2, ..., 2, 2, 2])

In [97]:
print ('accuracy ', accuracy_score(df_atheism_split['y_val'], prediction_SVC))
print ('precision ', precision_score(df_atheism_split['y_val'], prediction_SVC))
print (classification_report(df_atheism_split['y_val'], prediction_SVC))
print (confusion_matrix(df_atheism_split['y_val'], prediction_SVC))

accuracy  1.0
precision  0.0
              precision    recall  f1-score   support

           2       1.00      1.00      1.00    116040

    accuracy                           1.00    116040
   macro avg       1.00      1.00      1.00    116040
weighted avg       1.00      1.00      1.00    116040



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[[116040]]


In [86]:
cross_val_score(SVC_model, df_atheism_split['X_val'], df_atheism_split['y_val'], cv=5,scoring="accuracy")

/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:76: UserWarning: Label not 2 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:76: UserWarning: Label not 2 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:76: UserWarning: Label not 2 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:76: UserWarning: Label not 2 is present in all training examples.
  str(classes[c]))
/usr/local/lib/python3.6/dist-packages/sklearn/multiclass.py:76: UserWarning: Label not 2 is present in all training examples.
  str(classes[c]))


array([1., 1., 1., 1., 1.])

#### NB with fasttext - WIP

In [89]:
NB_pipeline = Pipeline([
                ("fasttext", model_fasttext_atheism),
                ("clf", OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])
                
NB_pipeline.fit(df_atheism_split['X_train'], df_atheism_split['y_train'])

TypeError: ignored

In [90]:
svc = Pipeline([("count_vectorizer", vectorizer), ("OneVSRest svc linear", OneVsRestClassifier(SVC(kernel='linear')))])
svc_tfidf = Pipeline([("tfidf_vectorizer", tf_vectorizer), ("OneVSRest svc linear", OneVsRestClassifier(SVC(kernel='linear')))])
#randomF = Pipeline([("count_vectorizer", vectorizer), ("RandomForestClassifier", RandomForestClassifier(n_estimators=100))])
#randomF_tfidf = Pipeline([("tfidf_vectorizer", tf_vectorizer), ("RandomForestClassifier", RandomForestClassifier(n_estimators=100))])
#extraT = Pipeline([("count_vectorizer", vectorizer), ("ExtraTreesClassifier", ExtraTreesClassifier(n_estimators=100))])
#extraT_tfidf = Pipeline([("tfidf_vectorizer", tf_vectorizer), ("ExtraTreesClassifier", ExtraTreesClassifier(n_estimators=100))])
etree_w2v = Pipeline([("word2vec vectorizer", MeanEmbeddingVectorizer(w2v)), ("word2vec extra trees", ExtraTreesClassifier(n_estimators=100))])
etree_w2v_tfidf = Pipeline([("tfidf word2vec vectorizer", TfidfEmbeddingVectorizer(w2v)), ("tfidf word2vec extra trees", ExtraTreesClassifier(n_estimators=100))])


all_models = [
    ("svc", svc),
    ("svc_tfidf", svc_tfidf),
    #("randomF", randomF), 
    #("randomF_tfidf", randomF_tfidf),
    #("extraT", extraT), 
    #("extraT_tfidf", extraT_tfidf),
    ("etree_w2v", etree_w2v), 
    ("etree_w2v_tfidf", etree_w2v_tfidf)
]
scores = sorted([(name, cross_val_score(model, df['text'], df['product'], cv=kfold).mean()) 
             for name, model in all_models], 
            key=lambda args: -(args[1]))

NameError: ignored